# CS247 Advanced Data Mining - Assignment 3
## Deadline: 11:59PM, February 7, 2023

## Instructions
Each assignment is structured as a Jupyter notebook, offering interactive tutorials that align with our lectures. You will encounter two types of problems: *write-up problems* and *coding problems*.

1. **Write-up Problems:** These problems are primarily theoretical, requiring you to demonstrate your understanding of lecture concepts and to provide mathematical proofs for key theorems. Your answers should include sufficient steps for the mathematical derivations.
2. **Coding Problems:** Here, you will be engaging with practical coding tasks. These may involve completing code segments provided in the notebooks or developing models from scratch.

To ensure clarity and consistency in your submissions, please adhere to the following guidelines:

* For write-up problems, use Markdown bullet points to format text answers. Also, express all mathematical equations using $\LaTeX$ and avoid plain text such as `x0`, `x^1`, or `R x Q` for equations.
* For coding problems, comment on your code thoroughly for readability and ensure your code is executable. Non-runnable code may lead to a loss of **all** points. Coding problems have automated grading, and altering the grading code will result in a deduction of **all** points.
* Your submission should show the entire process of data loading, preprocessing, model implementation, training, and result analysis. This can be achieved through a mix of explanatory text cells, inline comments, intermediate result displays, and experimental visualizations.

### Submission Requirements

* Submit your solutions through GradeScope in BruinLearn.
* Late submissions are allowed up to 24 hours post-deadline with a penalty factor of $\mathbf{1}(t\leq24)e^{-(\ln(2)/12)t}$.

### Collaboration and Integrity

* Collaboration is encouraged, but all final submissions must be your own work. Please acknowledge any collaboration or external sources used, including websites, papers, and GitHub repositories.
* Any suspicious cases of academic misconduct will be reported to The Office of the Dean of Students.

## Outline
* Part 1: Topic Model Continued
* Part 2: Word Embeddings

#### GPU Support

Considering the size of the training data, it is strongly suggested to use [Google Colab](https://colab.research.google.com/) or a GPU server for this exercise. If you are using Colab, you can manually switch to a CPU device on Colab by clicking `Runtime -> Change runtime type` and selecting `GPU` under `Hardware Accelerator`.

In [1]:
import torch

USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
elif USE_GPU and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [2]:
device

device(type='cpu')

## Part 1: Topic Models Continued (35 points)

Please see the attached PDF for the write-up problems.

## Part 2: Language Modeling (65 points + 20 bonus points)

### Section 1: Skip-Gram and Word2Vec (55 points + 10 bonus points)

The first section of this exercise is to implement and analyze a Skip-Gram model for language modeling, with a focus on understanding and experimenting with negative sampling techniques.

#### Environment Setup
Install the following packages:
* `torch`
* `torchdata`
* `torchtext`
* `portalocker`
* `nltk`

In [3]:
%pip install torchtext

In [4]:
%pip install -U nltk
%pip install -U portalocker

In [5]:
# Imports
import nltk
import numpy as np
import torch
import random
import torch.nn as nn
import torch.optim as optim
import collections

from tqdm import tqdm
from torchtext.datasets import AG_NEWS
from nltk.tokenize import word_tokenize
from torch.utils.data import DataLoader, Dataset

#### Exercise 1: Data Preprocessing (10 points)

Data preprocessing is a crucial step in any machine learning task. For the Skip-Gram model, the main goal is to convert raw text into a format that can be fed into the neural network. This involves loading the dataset, tokenizing the text, building a vocabulary, and generating context-target pairs.
* Tokenization: Convert the raw text into tokens (words).
* Vocabulary Building: Create a mapping of unique words to integers (word indices).
* Context-Target Pair Generation: For each word in the text, you will generate context-target pairs based on a specified window size. For example, with a window size of 2, for each word, the two words before and after become its context.

We will use the AG News dataset, a collection of news articles categorized into four classes. For our purpose, we are only interested in the textual content.
The following cell has already completed the first two steps for you. You will need to complete the third step to generate context-target pairs.

In [6]:
nltk.download('punkt')

class CustomDataset(Dataset):
    def __init__(self, texts, window_size=2):
        self.tokenized_texts = [word_tokenize(text.lower()) for text in texts]
        self.vocab = self.build_vocab(self.tokenized_texts)
        self.word_to_idx = {word: i for i, word in enumerate(self.vocab.keys())}
        self.idx_to_word = {i: word for word, i in self.word_to_idx.items()}
        self.data = self.generate_training_data(window_size)

    def build_vocab(self, texts):
        # Build a vocabulary with word frequencies for dataset
        vocab = collections.Counter()
        for text in texts:
            vocab.update(text)
        # Filter out words that appear less than 5 times
        vocab = {word: freq for word, freq in vocab.items() if freq > 5}
        return vocab

    def generate_training_data(self, window_size):
      training_data = []
      for line in self.tokenized_texts:
        for idx, token in enumerate(line):
            if token in self.vocab:
                start = min(max(idx - window_size, 0), len(line)-1)
                end = max(min(idx + window_size + 1, len(line)), 0)
                for context_idx in range(start, end):
                    context_token = line[context_idx]
                    if context_token in self.vocab and context_idx != idx:
                        training_data.append((self.word_to_idx[token], self.word_to_idx[context_token]))
      return training_data



    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        center_word, context_word = self.data[idx]
        return torch.tensor(center_word), torch.tensor(context_word)

news = AG_NEWS(split='test')
dataset = CustomDataset([text for _, text in news])
dataloader = DataLoader(dataset, batch_size=1024, shuffle=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
# Verify that the dataset is constructed as expected
for i, data in enumerate(dataset):
    center_word, context_word = data
    print(dataset.idx_to_word[center_word.item()], '->', dataset.idx_to_word[context_word.item()])
    if i >= 10:
        break

fears -> for
fears -> t
for -> fears
for -> t
t -> fears
t -> for
t -> pension
pension -> t
pension -> after
pension -> talks
after -> pension


#### Execrise 2 (5 points)
In the above code, we only consider very simple tokenization of the training corpus.
In fact, there are many other ways to tokenize the text, such as [stemming](https://en.wikipedia.org/wiki/Stemming) and [lemmatization](https://en.wikipedia.org/wiki/Lemmatization). Explain the reason why or why not we should use stemming and/or lemmatization in this task.

**[TODO: Write your answer here]**

The stemming and lemmatization can reduce the complexity of the text and size of vocabulary, but the choice really depends on the task and trade-offs between accuracy and computational efficiency. For tasks like training word embedding models where understanding of word can significantly impact the quality of the learned vectors, lemmatization would be more appropriate despites its higher computational cost.

#### Exercise 3: Model Implementation without Negative Sampling (4 points)

The Skip-Gram model takes a target word as input and tries to predict its context words.
In this exercise, please implement the Skip-Gram model using PyTorch. The model should have an [`torch.nn.embedding`](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html) layer to convert word indices to dense vectors and a [`torch.nn.linear`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) layer to predict context words.
Then, implement the `forward` function to define how data will pass through your model. It involves taking input words, transforming them through the embedding layer, and then making predictions using the linear layer.

In [8]:
class SkipGramModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(SkipGramModel, self).__init__()
        # TODO: Implement the Skip-Gram model architecture
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)


    def forward(self, center_words):
        out = None
        # TODO: Implement the forward pass of the Skip-Gram model
        out = self.linear(self.embeddings(center_words))
        return out

#### Exercise 4: Model Evaluation by Finding Similar Words (5 points)

Before implementing the training procedure, we can think about evaluating the model by finding similar words. The similarity between two words can be measured by the cosine similarity between their embeddings. The higher the cosine similarity, the more similar the two words are. For example, the cosine similarity between "cat" and "dog" should be higher than the cosine similarity between "cat" and "car".

In this exercise, you will implement a function to find the top-k most similar words to a given word. You will need to compute the cosine similarity between the given word and all other words in the vocabulary. Then, you will return the top-k words with the highest cosine similarity.

In [9]:
from torch.nn.functional import cosine_similarity

def find_most_similar(word, embedding_matrix, word_to_idx, idx_to_word, top_k=5):
    """
    Find the top_k most similar words to the given word based on cosine similarity
    :param word: The word to query
    :param embedding_matrix: The embedding matrix to query
    :param word_to_idx: The word-to-index mapping
    :param idx_to_word: The index-to-word mapping
    :param top_k: The number of most similar words to return

    :return: A list of (word, similarity) pairs of length top_k, sorted by the similarity in descending order
            Each pair denotes the most similar word to the given word and their cosine similarity.
    """
    if word not in word_to_idx:
        return f'Word {word} not in vocabulary!'

    word_idx = torch.tensor([word_to_idx[word]]).to(device)
    word_embedding = embedding_matrix(word_idx)
    similar_words = []
    # TODO: Calculate the cosine similarity between the query word and all words in the vocabulary
    similarity = cosine_similarity(embedding_matrix.weight, word_embedding)
    similarity[word_idx] = -1
    top_k_similarities, top_k_indices = torch.topk(similarity, top_k)
    similar_words = [(idx_to_word[index.item()], score.item()) for index, score in zip(top_k_indices, top_k_similarities)]


    return similar_words

After implementing the function, you can run the following cell to find the top-5 most similar words to "Basketball".
Note that since the model is not trained yet, the results should be random.

In [12]:
def evaluate(model, embedding_matrix, valid_word):
    similar_words = find_most_similar(valid_word, embedding_matrix, dataset.word_to_idx, dataset.idx_to_word)
    print(f"Model: {model}")
    print(f"Words similar to '{valid_word}':")
    for i, (word, similarity) in enumerate(similar_words):
        print(f'  {i + 1}. {word}: {similarity * 100:.2f}%')

embedding_dim = 100
vocab_size = len(dataset.vocab)
skipgram_model = SkipGramModel(vocab_size, embedding_dim).to(device)

evaluate(skipgram_model, skipgram_model.embeddings, 'basketball')

Model: SkipGramModel(
  (embeddings): Embedding(5748, 100)
  (linear): Linear(in_features=100, out_features=5748, bias=True)
)
Words similar to 'basketball':
  1. distraction: 40.49%
  2. yao: 34.81%
  3. semifinal: 31.39%
  4. past: 31.05%
  5. fuels: 30.18%


Besides word similarity, we can also quantitatively evaluate the model on some downstream tasks. Here, we adopt the WordSim-353 dataset, which contains 353 pairs of words and their similarity scores. The similarity scores are human-annotated and range from 0 to 10.
We can use the cosine similarity between the embeddings of two words as the predicted similarity score. Then, we can compute the Spearman's rank correlation coefficient between the predicted and the human-annotated similarity scores. The higher the Spearman's rank correlation coefficient, the better the model performs.

In [13]:
def get_embedding(word, embeddings, word_to_idx):
    idx = word_to_idx.get(word)
    if idx is not None:
        idx = torch.tensor([idx]).to(device)
        return embeddings(idx).detach().cpu().numpy()
    else:
        return None

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity as cos_sim
from scipy.stats import spearmanr

def wordsim(embedding_matrix):
    wordsim_data = pd.read_csv('/content/drive/My Drive/wordsim.txt', sep='\t', header=None, names=['Word 1', 'Word 2', 'Similarity'])

    # Calculate cosine similarities
    cosine_similarities = []
    for _, row in wordsim_data.iterrows():
        word1, word2, human_score = row['Word 1'], row['Word 2'], row['Similarity']
        embedding1 = get_embedding(word1, embedding_matrix, dataset.word_to_idx)
        embedding2 = get_embedding(word2, embedding_matrix, dataset.word_to_idx)

        if embedding1 is not None and embedding2 is not None:
            sim = cos_sim(embedding1, embedding2)[0][0]
            cosine_similarities.append((human_score, sim))

    # Calculate Spearman's rank correlation
    human_scores, model_scores = zip(*cosine_similarities)
    correlation, _ = spearmanr(human_scores, model_scores)
    print(f'Spearman correlation: {correlation}')

wordsim(skipgram_model.embeddings)

Spearman correlation: 0.14446807022795716


#### Exercise 5: Model Training (5 points)

Implement the `train_skipgram` function to train the model. The function should take a batch of context-target pairs as input and return the loss. You will need to compute the objective function using the negative log-likelihood loss. Then, you will need to update the model parameters using the optimizer.
Note that in PyTorch, [`CrossEntropyLoss` combines `LogSoftmax` and `NLLLoss` in a single class](https://towardsdatascience.com/cross-entropy-negative-log-likelihood-and-all-that-jazz-47a95bd2e81). Therefore, you can directly use `CrossEntropyLoss` to compute the negative log-likelihood loss.

Here we use the [Adam optimizer](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam) with a learning rate of 0.01. During last week's discussions, we mentioned the importance of choosing appropriate optimizers and learning rates. In practice, the Adam optimizer is a good choice for many tasks, while SGD can lead to better convergence in some cases. However, the choice of learning rate can be more tricky. In general, a learning rate that is too small will lead to slow convergence, while a learning rate that is too large will lead to divergence. In this exercise, you can use the default learning rate of 0.01. However, you are encouraged to experiment with different optimizers and/or learning rates and observe the impact on the training process.

For every 5 epochs, you will need to evaluate the model by finding the top-5 most similar words to "Basketball". You should observe that as the training progresses, the model is able to find more relevant words for "Basketball". On a GPU server, it should take about 10 minutes to finish training for 50 epochs.

In [21]:
learning_rate = 0.01
optimizer = optim.Adam(skipgram_model.parameters(), lr=learning_rate)

In [ ]:
def train_skipgram(model, data_loader, epochs, optimizer):
    # TODO: Define the loss function
    criterion = None
    criterion = nn.CrossEntropyLoss()

    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for data in data_loader:
            center_word, context_word = data
            center_word, context_word = center_word.to(device), context_word.to(device)

            # TODO: Implement the training pass for the Skip-Gram model
            # Zero the gradients
            optimizer.zero_grad()
            loss = criterion(model(center_word), context_word)
            loss.backward()
            optimizer.step()
            # Accumulate the loss
            total_loss += loss.item()
        if (epoch + 1) % 5 == 0:
            print(f'Epoch {epoch + 1}, Loss: {total_loss}')
            evaluate(model, model.embeddings, 'basketball')

train_skipgram(skipgram_model, dataloader, epochs=50, optimizer=optimizer)
wordsim(skipgram_model.embeddings)

Epoch 5, Loss: 5824.100010871887
Model: SkipGramModel(
  (embeddings): Embedding(5748, 100)
  (linear): Linear(in_features=100, out_features=5748, bias=True)
)
Words similar to 'basketball':
  1. soccer: 51.49%
  2. football: 46.59%
  3. olympic: 44.73%
  4. bekele: 43.13%
  5. boxing: 41.78%
Epoch 10, Loss: 5713.541437625885
Model: SkipGramModel(
  (embeddings): Embedding(5748, 100)
  (linear): Linear(in_features=100, out_features=5748, bias=True)
)
Words similar to 'basketball':
  1. soccer: 50.04%
  2. football: 49.53%
  3. volleyball: 44.56%
  4. olympic: 44.16%
  5. invitational: 39.38%
Epoch 15, Loss: 5681.682966709137
Model: SkipGramModel(
  (embeddings): Embedding(5748, 100)
  (linear): Linear(in_features=100, out_features=5748, bias=True)
)
Words similar to 'basketball':
  1. volleyball: 50.49%
  2. football: 50.12%
  3. soccer: 48.29%
  4. top-ranked: 40.57%
  5. olympic: 38.00%
Epoch 20, Loss: 5668.499978065491
Model: SkipGramModel(
  (embeddings): Embedding(5748, 100)
  (li

#### Exercise 6: Model Implementation with Negative Sampling (10 points)

Negative sampling is a technique to reduce the computation time in training by altering the objective to distinguish a target word from randomly sampled "negative" words instead of predicting the next word directly.

In this exercise, you will modify the earlier Skip-Gram model to incorporate negative sampling.
The new model should have two embedding layers: one for the target words and one for the context words.
For negative sampling, we consider the simplest case where the number of negative samples is fixed to 10.
For each target word, we will randomly sample 10 words from the vocabulary as negative samples.
Then, implement the `forward` function to define how data will pass through your model. It involves taking input words, transforming them through the embedding layers, and then computing the loss as the following equation:
$$
\mathcal{L} = -\log\sigma(\mathbf{u}_t^\top\mathbf{v}_c) - \sum_{i=1}^k\log\sigma(-\mathbf{u}_t^\top\mathbf{v}_{n_i}),
$$
where $\mathbf{u}_t$ is the embedding of the center word, $\mathbf{v}_c$ is the embedding of the context word (positive sample), $\mathbf{v}_{n_i}$ is the embedding of the $i$-th negative sample, and $\sigma$ is the sigmoid function.

In [ ]:
import torch.nn.functional as F

class SkipGramNegSampling(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(SkipGramNegSampling, self).__init__()
        # TODO: Define the Embedding layers for center words and context words
        self.center_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.context_embeddings = nn.Embedding(vocab_size, embedding_dim)

    def forward(self, center_words, context_words, negative_words):
        """
        Skip-Gram with Negative Sampling
        :param center_words: the center words of shape [batch_size]
        :param context_words: the context words of shape [batch_size]
        :param negative_words: the negative words of shape [batch_size, num_negative_samples]
        :return: the loss, a PyTorch scalar
        """
        # TODO: Implement the forward pass for Skip-Gram with Negative Sampling
        center_embeds = self.center_embeddings(center_words)
        context_embeds = self.context_embeddings(context_words)
        negative_embeds = self.context_embeddings(negative_words)

        loss = None
        positive_scores = torch.bmm(context_embeds.unsqueeze(1), center_embeds.unsqueeze(2)).squeeze()
        negative_scores = torch.bmm(negative_embeds, center_embeds.unsqueeze(2)).squeeze() * -1
        loss = -(F.logsigmoid(positive_scores).sum() + F.logsigmoid(negative_scores).sum())

        return loss

# Negative samples can be generated in various ways, here's a simple random approach
def get_negative_samples(batch_size, vocab_size, num_negative_samples):
    return torch.randint(vocab_size, (batch_size, num_negative_samples))

Now, you should follow the same training procedure as before. However, you will need to modify the `train_with_neg_sampling` function to generate context-target pairs with negative samples. You will also need to modify the `train_with_neg_sampling` function to compute the loss using the new objective function.

In [ ]:
def train_with_neg_sampling(model, data_loader, optimizer, epochs, vocab_size, num_negative_samples):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for center, context in data_loader:
            negative = get_negative_samples(center.size(0), vocab_size, num_negative_samples).to(device)
            center, context = center.to(device), context.to(device)

            # TODO: Implement the training pass for the Skip-Gram model with Negative Sampling
            optimizer.zero_grad()
            loss = model(center, context, negative)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        if (epoch + 1) % 5 == 0:
            print(f'Epoch {epoch + 1}, Loss: {total_loss}')
            evaluate(model, model.center_embeddings, 'basketball')

In [ ]:
neg_sampling_model = SkipGramNegSampling(vocab_size, embedding_dim).to(device)
optimizer = optim.Adam(neg_sampling_model.parameters(), lr=learning_rate)
train_with_neg_sampling(neg_sampling_model, dataloader, optimizer, epochs=50, vocab_size=vocab_size, num_negative_samples=10)
wordsim(neg_sampling_model.center_embeddings)

Epoch 5, Loss: 1728214.4376831055
Model: SkipGramNegSampling(
  (center_embeddings): Embedding(5748, 100)
  (context_embeddings): Embedding(5748, 100)
)
Words similar to 'basketball':
  1. league: 52.03%
  2. star: 50.54%
  3. khan: 47.98%
  4. team: 47.86%
  5. ernie: 47.15%
Epoch 10, Loss: 1570621.0600738525
Model: SkipGramNegSampling(
  (center_embeddings): Embedding(5748, 100)
  (context_embeddings): Embedding(5748, 100)
)
Words similar to 'basketball':
  1. league: 49.16%
  2. violating: 48.45%
  3. louisville: 45.80%
  4. soccer: 45.80%
  5. university: 45.16%
Epoch 15, Loss: 1530985.989944458
Model: SkipGramNegSampling(
  (center_embeddings): Embedding(5748, 100)
  (context_embeddings): Embedding(5748, 100)
)
Words similar to 'basketball':
  1. football: 50.72%
  2. university: 49.91%
  3. louisville: 48.36%
  4. soccer: 46.20%
  5. responsibility: 45.50%
Epoch 20, Loss: 1514766.8057250977
Model: SkipGramNegSampling(
  (center_embeddings): Embedding(5748, 100)
  (context_embeddi

#### Exercise 7: Exploring Different Negative Distributions in Word2Vec (16 points + 10 bonus points)

Negative sampling is a crucial technique in training word embeddings. It involves selecting "negative" samples (words not in the target context) to train the model more efficiently. The choice of distribution from which these negative samples are drawn can significantly affect the quality of the learned embeddings.

Your task is to experiment with different negative sampling distributions and analyze their impact on the model's performance. Below are several distributions you should consider:
- Frequency-based Distribution: Words are sampled according to their frequency in the corpus. Common approaches include sampling words with a probability proportional to their frequency or their frequency raised to 3/4th power (as used in Word2Vec).
- Subsampling of Frequent Words: Modify the frequency-based distribution to reduce the representation of extremely frequent words. This can help in reducing the bias towards very common words in the corpus.
- Custom Distribution based on Part-of-Speech Tags: Create a distribution where the probability of a word being sampled as negative depends on its [part-of-speech tag](https://www.nltk.org/book/ch05.html). For instance, you might choose to sample nouns or verbs more frequently.
- Contextual Similarity-Based Distribution: Develop a distribution that takes into account the similarity of words to the target word. Words that are contextually similar to the target word could have a lower probability of being selected as negative samples.

Please implement at least two of the above distributions and compare their performance with the uniform distribution. You can reuse the `train_with_neg_sampling` function to train the model with different negative sampling distributions. Then, please use the `wordsim` function to evaluate the model on the WordSim-353 dataset. Last, briefly summarize your findings and provide possible explanations for the observed results.

For students interested in further exploration, consider the following optional directions (10 bonus points):
- Combining Multiple Distributions: Explore the effects of using a hybrid approach, combining elements from different distributions.
- Dynamic Sampling Strategies: Investigate sampling strategies that change during training, perhaps based on the convergence rate or other learning dynamics.
- Corpus-Specific Distributions: Tailor the negative sampling distribution to the specific characteristics of the corpus you are using (e.g., technical vs. general language).
- Investigating the Impact on Semantic vs. Syntactic Tasks: Analyze how different sampling distributions affect the model's performance on semantic versus syntactic tasks.

You can also explore other negative sampling distributions that are not listed above. Please explain your findings and provide possible explanations for the observed results.

**[TODO: Write your answer here]**
1. Frequency-based Distribution:
In the frequency-based distribution, words are sampled according to their frequency in the corpus. We can sample words with a probability proportional to their frequency or their frequency raised to a power (e.g., 3/4th power, as used in Word2Vec).

The intuition behind this distribution is that rare words may not provide enough training signal, while very frequent words may dominate the embeddings and not capture enough meaningful information. By sampling negative samples based on their frequency, we can balance the importance of rare and frequent words in the training process.

2. Contextual Similarity-Based Distribution:
In the contextual similarity-based distribution, we consider the similarity of words to the target word. Words that are contextually similar to the target word could have a lower probability of being selected as negative samples.

To implement this distribution, we need a pre-trained model that can compute contextual word embeddings. We can use a model like BERT or GPT to encode the target context and negative samples into embeddings. Then, we can calculate the cosine similarity between the target context and negative samples to assign probabilities for sampling.

For evaluation, use the WordSim-353 dataset, which contains word pairs annotated with similarity scores. compare the Spearman correlation between the model's similarity scores and the ground truth scores for each distribution.

Here's an outline of the steps:
1. Implement the Frequency-based Distribution and train the model with this distribution.
2. Implement the Contextual Similarity-Based Distribution and train the model with this distribution.
3. Train the model with the uniform distribution as a baseline.
4. Evaluate the model's performance on the WordSim-353 dataset using the wordsim function.
5. Compare the Spearman correlation for each distribution and analyze the results.

### Section 2: GloVe Embeddings (10 points + 10 bonus points)

The second section of this exercise is to leverage pretrained GloVe embeddings to solve the word analogy task.

#### Exercise 8: Word Analogy Task (10 points)

In the word analogy task, we are given three words 'a', 'b', and 'c', and we need to find a word 'd' such that the relationship between 'a' and 'b' is similar to that between 'c' and 'd'. For example, in the analogy "man is to king as woman is to queen", the relationship between "man" and "king" is similar to that between "woman" and "queen".

To solve this, you will use pretrained GloVe embeddings. The general idea is to find the vector representing the relationship between 'a' and 'b', and then search for a word whose relationship with 'c' is most similar to this vector. This similarity is typically measured using cosine similarity.
Please implement the `word_analogy` function to find the word 'd' that best completes the analogy.

In [ ]:
from torchtext.vocab import GloVe
import numpy as np

# Load Pretrained GloVe Embeddings
glove = GloVe(name='6B', dim=100)  # 100-dimensional vectors

def word_analogy(a, b, c, embeddings):
    """
    Given words 'a', 'b', 'c', find 'd' such that 'a' is to 'b' as 'c' is to 'd'.
    Use cosine similarity to find the closest word.

    :param a, b, c: Words in the analogy a:b :: c:d
    :param embeddings: Pretrained GloVe embeddings
    :return: The word 'd'
    """

    # Get vectors for the input words
    vec_a = embeddings[a]
    vec_b = embeddings[b]
    vec_c = embeddings[c]

    # TODO: Compute the vector for 'd' (vec_a to vec_b is as vec_c to vec_d)
    vec_d = vec_b - vec_a + vec_c
    max = -np.inf
    # TODO: Search for the word that is closest to vec_d
    # Hint: Iterate over the embeddings and use cosine_similarity
    d = None

    for word in embeddings.stoi:
        if word in [a, b, c]:
            continue

        vec_word = embeddings[word]
        similarity = torch.nn.functional.cosine_similarity(vec_d.unsqueeze(0), vec_word.unsqueeze(0))

        if similarity > max:
            max = similarity
            d = word

    return d

.vector_cache\glove.6B.zip: 862MB [02:38, 5.43MB/s]                              
100%|█████████▉| 399999/400000 [00:18<00:00, 21525.10it/s]


Below are examples to test the word analogy function:
- Geographical Analogies: "France is to Paris as Italy is to ?"
- Grammatical Analogies: "happy is to happiest as cold is to ?"
- Semantic Analogies: "cat is to kitten as dog is to ?"
- Syntactic Analogies: "quick is to quickly as slow is to ?"

In [ ]:
test_cases = [['france', 'paris', 'italy'], ['happy', 'happiest', 'cold'], ['cat', 'kitten', 'dog'], ['quick', 'quickly', 'slow']]

for a, b, c in test_cases:
    d = word_analogy(a, b, c, glove)
    print(f'{a} is to {b} as {c} is to {d}')

france is to paris as italy is to rome
happy is to happiest as cold is to frigid
cat is to kitten as dog is to puppy
quick is to quickly as slow is to rapidly


### Section 3: Debiasing Word Vectors (10 bonus points)

Bias in word vectors is a significant issue in natural language processing, leading to stereotypical or prejudiced results in various applications. The paper [*Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings*](https://arxiv.org/abs/1607.06520) presents methods for reducing gender bias in word embeddings. This exercise focuses on implementing two key algorithms from this paper: Neutralization and Equalization.
- Neutralization: Remove gender bias from non-gender specific words.
- Equalization: Adjust vectors for gender-specific words to ensure equal gender representation.

To show how GloVe word embeddings relate to gender, let's examine the relationship between occupation-related words in GloVe embeddings and a constructed "gender" vector.
The "gender" vector is defined as the difference between the GloVe embeddings of several gender-related word pairs, such as "man - woman", "king - queen", "mother - father", "girl - boy", etc.

In [ ]:
def construct_gender_vector(embeddings, gender_pairs):
    """
    Construct a gender vector based on differences of word pairs.

    :param embeddings: GloVe word embeddings.
    :param gender_pairs: List of tuples containing gender-specific word pairs.
    :return: Constructed gender vector.
    """
    gender_vector = []
    for word_a, word_b in gender_pairs:
        gender_vector.append(embeddings[word_a] - embeddings[word_b])
    return np.mean(gender_vector, axis=0)

gender_pairs = [('girl', 'boy')]
gender_vector = construct_gender_vector(glove, gender_pairs)

Then, we can calculate the cosine similarity between the gender vector and the GloVe embeddings of some common occupation-related words.
It can be observed that these similarities indeed reflect certain unhealthy gender stereotypes and biases.

In [ ]:
def similarity_with_gender(word, embeddings, gender_vector):
    """
    Calculate cosine similarity between a word and the gender vector.

    :param word: Target word.
    :param embeddings: GloVe word embeddings.
    :param gender_vector: Constructed gender vector.
    :return: Cosine similarity.
    """
    word_vector = embeddings[word]
    similarity = cosine_similarity(word_vector, torch.from_numpy(gender_vector).unsqueeze(0))
    return similarity

occupation_words = ['nurse', 'engineer', 'teacher', 'scientist', 'receptionist', 'programmer', 'actor', 'doctor']
for word in occupation_words:
    similarity = similarity_with_gender(word, glove, gender_vector)
    print(f"Similarity of '{word}' with gender vector: {similarity}")

Similarity of 'nurse' with gender vector: tensor([0.1591])
Similarity of 'engineer' with gender vector: tensor([-0.1748])
Similarity of 'teacher' with gender vector: tensor([0.0444])
Similarity of 'scientist' with gender vector: tensor([-0.1127])
Similarity of 'receptionist' with gender vector: tensor([0.2157])
Similarity of 'programmer' with gender vector: tensor([-0.1941])
Similarity of 'actor' with gender vector: tensor([0.0319])
Similarity of 'doctor' with gender vector: tensor([-0.0178])


#### Exercise 9: Neutralization (5 bonus points)

For a word vector $\mathbf{w}$ and a gender direction $\mathbf{g}$, the neutralization is performed by removing the projection of $\mathbf{w}$ on $\mathbf{g} $. The equation for neutralization is:
$$
\mathbf{w}_{\text{neutralized}} = \mathbf{w} - \frac{\mathbf{w} \cdot \mathbf{g}}{\|\mathbf{g}\|^2} \mathbf{g}.
$$

You task is to implement the Neutralization algorithm to remove gender bias from non-gender specific words.

In [ ]:
def neutralize(word_vector, gender_direction):
    """
    Neutralize bias in the word vector by removing the projection on the gender direction.

    :param word_vector: Vector representation of the word.
    :param gender_direction: The gender direction vector.
    :return: Debiased word vector.
    """
    # TODO: Implement this function to neutralize the word vector
    project = np.dot(word_vector, gender_direction) / np.dot(gender_direction, gender_direction)
    return word_vector - project * gender_direction

word_vector = glove['receptionist']
print('Before neutralization, the similarity of "receptionist" with gender vector:', cosine_similarity(word_vector, torch.from_numpy(gender_vector).unsqueeze(0)))
neutralized_vector = neutralize(word_vector, gender_vector)
print('After neutralization, the similarity of "receptionist" with gender vector:', cosine_similarity(neutralized_vector, torch.from_numpy(gender_vector).unsqueeze(0)))

Before neutralization, the similarity of "receptionist" with gender vector: tensor([0.2157])
After neutralization, the similarity of "receptionist" with gender vector: tensor([-5.5879e-09])


#### Exercise 10: Equalization (5 bonus points)

The Equalization algorithm is designed to adjust pairs of gender-specific words so that they are equidistant to the gender-neutral words.
The procedure involves several steps, including computing the mean of the pair, projecting onto the gender direction, and normalizing the difference.

Given a pair of word vectors $\mathbf{w}_a$ and $\mathbf{w}_b$ for gender-specific words (e.g., 'actor' and 'actoress'), and the gender direction $\mathbf{g}$, the equalization adjusts both vectors so that they are equidistant to the bias direction. The key steps involve:

1. Calculate the mean vector $$\mu = \frac{\mathbf{w}_a + \mathbf{w}_b}{2}.$$
2. Compute the projections of $\mu$ over the gender direction:
   $$
   \begin{align}
      \mu_B &= \frac{\langle \mu, \mathbf{g} \rangle}{\| \mathbf{g} \|^2} \mathbf{g}, \\
      \mu_{\perp} &= \mu - \mu_B.
   \end{align}
   $$
3. Adjust the bias parts of $\mathbf{w}_{aB}$ and $\mathbf{w}_{bB}$:
   $$
   \begin{align}
      \mathbf{w}_{aB}^{\text{corrected}} &= \sqrt{ \left| 1 - \| \mu_{\perp} \|^2 \right| } \times \frac{\mathbf{w}_{aB} - \mu_B}{|(\mathbf{w}_{a} - \mu_{\perp}) - \mu_B|}, \\
      \mathbf{w}_{bB}^{\text{corrected}} &= \sqrt{ \left| 1 - \| \mu_{\perp} \|^2 \right| } \times \frac{\mathbf{w}_{bB} - \mu_B}{|(\mathbf{w}_{b} - \mu_{\perp}) - \mu_B|}.
   \end{align}
   $$
4. Combine the corrected bias parts with $\mu_{\perp}$:
   $$
   \begin{align}
     \mathbf{w}_{a}^{\text{corrected}} &= \mathbf{w}_{aB}^{\text{corrected}} + \mu_{\perp}, \\
     \mathbf{w}_{b}^{\text{corrected}} &= \mathbf{w}_{bB}^{\text{corrected}} + \mu_{\perp}.
   \end{align}
   $$

In [ ]:
def normalize_vector(vector):
    """
    Normalize a given vector.
    :param vector: Input vector to be normalized.
    :return: Normalized vector.
    """
    return vector / torch.norm(vector)

def equalize(w_a, w_b, g):
    """
    Equalize the word vectors for a pair of gender-specific words.

    :param w_a, w_b: Word vectors for the gender-specific pair.
    :param g: The gender direction vector.
    :return: Equalized word vectors for the pair.
    """
    # TODO: Implement this function to equalize the word vectors
    if isinstance(w_a, np.ndarray):
        w_a = torch.tensor(w_a, dtype=torch.float32)
    if isinstance(w_b, np.ndarray):
        w_b = torch.tensor(w_b, dtype=torch.float32)
    if isinstance(g, np.ndarray):
        g = torch.tensor(g, dtype=torch.float32)

    g = g / torch.norm(g)

    mu_a = torch.dot(w_a, g) / torch.dot(g, g)
    mu_b = torch.dot(w_b, g) / torch.dot(g, g)

    mu_avg = (mu_a + mu_b) / 2

    w_a_corrected = w_a - g * (mu_a - mu_avg)
    w_b_corrected = w_b - g * (mu_b - mu_avg)

    return normalize_vector(w_a_corrected), normalize_vector(w_b_corrected)

In [ ]:
w_a, w_b = glove['actor'], glove['actress']
equalized_vectors = equalize(w_a, w_b, gender_vector)
print('Cosine similarity before equalization:')
print(f'  {cosine_similarity(w_a, torch.from_numpy(gender_vector).unsqueeze(0))} (actor)')
print(f'  {cosine_similarity(w_b, torch.from_numpy(gender_vector).unsqueeze(0))} (actress)')
print('Cosine similarity after equalization:')
print(f'  {cosine_similarity(equalized_vectors[0], torch.from_numpy(gender_vector).unsqueeze(0))} (actor)')
print(f'  {cosine_similarity(equalized_vectors[1], torch.from_numpy(gender_vector).unsqueeze(0))} (actress)')

Cosine similarity before equalization:
  tensor([0.0319]) (actor)
  tensor([0.4012]) (actress)
Cosine similarity after equalization:
  tensor([0.2257]) (actor)
  tensor([0.2290]) (actress)
